In [1]:
from pathlib import Path

docs = []
for file in Path("../../docs").glob("*.txt"):
    try:
        with open(file, encoding="utf-8", errors="ignore") as file_handler:
            docs.append(file_handler.read())
    except FileNotFoundError:
        print(file)

..\..\docs\Democratize computer vision defect detection for manufacturing quality using no-code machine learning with Amazon SageMaker Canvas _ AWS Machine Learning Blog.txt
..\..\docs\Helping Customers Modernize Their Cloud Infrastructure Using the AWS Well-Architected Framework with Comprinno _ Comprinno Technologies Case Study _ AWS.txt
..\..\docs\Isetan Mitsukoshi System Solutions seamlessly migrates databases to Amazon Aurora using Amazon DMA _ Isetan Mitsukoshi System Solutions Case Study _ AWS.txt
..\..\docs\NTT DOCOMO builds a new data analysis platform for 9000 workers with AWS attracting 13 times more users and invigorating data use _ NTT Docomo Case Study _ AWS.txt
..\..\docs\Transforming fleet telematics into predictive analytics with Capgeminis Trusted Vehicle and AWS IoT FleetWise _ The Internet of Things on AWS  Official Blog.txt
..\..\docs\WebBeds uses Amazon EC2 Spot Instances to save its business amid a reduction in travel worldwide and reduce costs up to 64 percent. 

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
collection_name = "aws-case-studies-and-blogs"
vector_params = models.VectorParams(
    size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by the model
    distance=models.Distance.COSINE,
)

if qdrant.collection_exists(collection_name):
    qdrant.delete_collection(collection_name)

qdrant.create_collection(collection_name=collection_name, vectors_config=vector_params)

True

In [6]:
# vectorize!
vectors = encoder.encode(docs)
qdrant.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"text": doc},
        )
        for idx, (vector, doc) in enumerate(
            zip(vectors, docs)
        )  # data is the variable holding all the wines
    ],
)

In [7]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [8]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'text': '\n\n\n\nAWS Machine Learning Blog\n\n\n\n\n\n\n\n\nIntroducing popularity tuning for Similar-Items in Amazon Personalize\n\n\n\n        by \n       \nJulia Clark\n, \n       \nBranislav Kveton\n, \n       \nNihal Harish\n, and \n       \nYifei Ma\n | on \n       \n08 JUN 2023\n | in \n       \nAmazon Machine Learning\n, \nAmazon Personalize\n | \n       \nPermalink\n | \n       \n Comments\n | \n       \n\xa0Share\n\n\n\n\n\n\n \n \n\n\n \n \n\n\n \n \n\n\n \n \n\n\n\n\n\n\n\n\n\n\n\n\nAmazon Personalize\n now enables popularity tuning for its \nSimilar-Items recipe\n (\naws-similar-items\n). Similar-Items generates recommendations that are similar to the item that a user selects, helping users discover new items in your catalog based on the previous behavior of all users and item metadata. Previously, this capability was only available for \nSIMS\n, the other \nRelated_Items\n recipe within Amazon Personalize.\n\n\nEvery customer’s item catalog and the way that users interac

In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
# Enter a query
query = "Write a detailed summary of AWS Personalize and in your response provided an executive summary after which you can present succinct bullet points? Remember to add formatting elements so that the output is easy to read and well-formatted. Use line breaks to improve formatting."
# query = "Write a detailed summary of the use of Machine Learning by AWS customers in the Life Science industry. In your response provided an executive summary after which you can present succinct bullet points? Remember to add formatting elements so that the output is easy to read and well-formatted. Use line breaks to improve formatting."
# query = "Write a detailed summary of the companies using AWS IoT services. In your response provided an executive summary after which you can present succinct bullet points? Remember to add formatting elements so that the output is easy to read and well-formatted. Use line breaks to improve formatting."
# query = "Write a detailed summary of the competitive advantage of AWS services over other cloud vendors in the AI space. In your response provided an executive summary after which you can present succinct bullet points? Remember to add formatting elements so that the output is easy to read and well-formatted. Use line breaks to improve formatting."
# query = ""Write a detailed summary how GenAI applications can be used in the Fintech industry. In your response provided an executive summary after which you can present succinct bullet points? Remember to add formatting elements so that the output is easy to read and well-formatted. Use line breaks to improve formatting.""

In [12]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot. Your priority is to help guide users."},
        {"role": "user", "content": query},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='The article "Novo Nordisk: Using AWS to streamline manufacturing processes and reduce manual labor" by the author provides an overview of the company\'s journey to adopting AWS for its manufacturing operations. The article highlights the company\'s use of AWS services such as AWS IoT Greengrass, AWS S3, and AWS SageMaker Edge, which have enabled them to automate key tasks on manufacturing lines, enhancing quality, and improving yield. The article also discusses the company\'s use of AWS IoT Greengrass to automate key tasks on manufacturing lines, including cartridge counting and anomaly detection for agar plates. The article concludes by highlighting the benefits of using AWS for Novo Nordisk\'s manufacturing operations, including increased efficiency, reduced manual labor, and improved quality.</s>', role='assistant', function_call=None, tool_calls=None)
